In [1]:
from llama_index.schema import Node, NodeWithScore
from llama_index.response_synthesizers import get_response_synthesizer

response_synthesizer = get_response_synthesizer(response_mode='compact')

In [2]:
%cd '/home/pedram/projects/llm-support-bot/llm-support-bot' 
import storage as s

/home/pedram/projects/llm-support-bot/llm-support-bot
INFO:chromadb.telemetry.posthog:Anonymized telemetry enabled. See https://docs.trychroma.com/telemetry for more information.
Anonymized telemetry enabled. See https://docs.trychroma.com/telemetry for more information.


In [3]:
from llama_index.llms import OpenAI
from llama_index import ServiceContext
from llama_index.llms import LlamaCPP
from llama_index.llms.llama_utils import messages_to_prompt, completion_to_prompt
from langchain.embeddings.huggingface import HuggingFaceEmbeddings
from llama_index import get_response_synthesizer, set_global_service_context

# model_url = 'https://huggingface.co/TheBloke/CodeLlama-13B-Instruct-GGUF/resolve/main/codellama-13b-instruct.Q4_K_M.gguf'
model_url = 'https://huggingface.co/TheBloke/CodeLlama-13B-Instruct-GGUF/resolve/main/codellama-13b-instruct.Q6_K.gguf'
llm = LlamaCPP(
    model_url=model_url,
    temperature=0.1,
    max_new_tokens=2000,
    context_window=3900,
    generate_kwargs={},
    model_kwargs={"n_gpu_layers": -1},
    messages_to_prompt=messages_to_prompt,
    completion_to_prompt=completion_to_prompt,
    verbose=False,
)

ggml_init_cublas: found 1 CUDA devices:
  Device 0: NVIDIA GeForce RTX 4080, compute capability 8.9
llama_model_loader: loaded meta data with 20 key-value pairs and 363 tensors from /home/pedram/projects/llm-cache/models/codellama-13b-instruct.Q6_K.gguf (version GGUF V2 (latest))
llama_model_loader: - tensor    0:                token_embd.weight q6_K     [  5120, 32016,     1,     1 ]
llama_model_loader: - tensor    1:           blk.0.attn_norm.weight f32      [  5120,     1,     1,     1 ]
llama_model_loader: - tensor    2:            blk.0.ffn_down.weight q6_K     [ 13824,  5120,     1,     1 ]
llama_model_loader: - tensor    3:            blk.0.ffn_gate.weight q6_K     [  5120, 13824,     1,     1 ]
llama_model_loader: - tensor    4:              blk.0.ffn_up.weight q6_K     [  5120, 13824,     1,     1 ]
llama_model_loader: - tensor    5:            blk.0.ffn_norm.weight f32      [  5120,     1,     1,     1 ]
llama_model_loader: - tensor    6:              blk.0.attn_k.weight q6_

In [4]:
embed_model = s.get_embedding_model()
service_context = ServiceContext.from_defaults(llm=llm, embed_model=embed_model)
set_global_service_context(service_context)

In [27]:
index = s.load_index()
response_synthesizer = get_response_synthesizer(response_mode='compact')

In [28]:
from llama_index.prompts import PromptTemplate

template = (
    "We have provided Github Issues and Discussions as context below. \n"
    "---------------------\n"
    "{context_str}"
    "\n---------------------\n"
     """
You are a support bot, searching for relevant information in Github Issues and Discussions.
You will receive a question from the user, and will try to find relevant information.
If you find a relevant result, include the type, name, and URL of the document, as well as a possible solution to their problem.
An Example Answer:

Summary: (summarize the question)
Relevant Issues: (link to the issues or discussions)
(Your Answer below)
   
Given this information, please answer the question: {query_str}\n"""
)
qa_template = PromptTemplate(template)
query_engine = index.as_query_engine(service_context=service_context, response_synthesizer=response_synthesizer, text_qa_template=qa_template)

In [29]:
question="""
I have successfully integrated a dbt model into Dagster. Description and metadata are correctly pulled in by Dagster for my tables and visible both in the graph (the description) and the corresponding asset detail pages . However, source details do not seem to be populated when I click on the asset details. The model runs nicely via Dagster and sources appear in the graph.

My dummy dbt schema.yml is as follows:

version: 2

models:
  - name: test_model_1
    description: "This model contains user-level revenue data aggregated at the monthly level."
    columns:
      - name: n_of_rows
    config:
      meta:
        dagster:
          group: revenue

sources:
  - name: external
    description: "This is an external source"
    database: ext
    schema: ext
    tables:
    - name: payments
      description: "User payment data"
      meta:
        dagster:
          group: revenue
"""

In [30]:
response = query_engine.query(question)

In [31]:
print(response)

  Thank you for reaching out! I'm happy to help you with your question. However, I must point out that the context information provided does not seem to be relevant to the query you have asked. The context information is about a different issue related to automatic asset reconciliation in Dagster, while your query seems to be about a problem with source details not being populated when clicking on an asset detail page in Dagster.

To better assist you, could you please provide more information or clarify the question? For example, what is the exact error message you are seeing, and what have you tried so far to resolve the issue? Additionally, it would be helpful if you could provide a minimal reproducible example of your code, including any relevant configuration files. This will allow me to better understand the problem and provide a more accurate answer.
